# Putting it all together (PyTorch)

In [ ]:

#  Install the Transformers, Datasets, and Evaluate libraries to run this notebook.
!pip install datasets evaluate transformers[sentencepiece]

In [2]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

Как можно токенизировать

In [7]:
# Дополняет до самой длинной последовательности в батче 
# model_inputs = tokenizer(sequences, padding="longest")
# Но что эквивалентно 
model_inputs = tokenizer(sequences, padding=True)

# Дополняет до заданной max_length
# model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
# Will pad the sequences up to the model max length (512 for BERT or DistilBERT)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [ ]:
# преобразование в определенные тензоры фреймворка
# model_inputs = tokenizer(sequences, padding=True, return_tensors="pt") # Returns PyTorch tensors
# model_inputs = tokenizer(sequences, padding=True, return_tensors="tf") # Returns TensorFlow tensors
# model_inputs = tokenizer(sequences, padding=True, return_tensors="np") # Returns NumPy arrays

Декодирование тензоров

In [9]:
decoded_texts = tokenizer.batch_decode(model_inputs["input_ids"], skip_special_tokens=True)
decoded_texts

["i've been waiting for a huggingface course my whole life.", 'so have i!']

Итого, обрабатываем  
- несколько последовательностей (с выравниванием `padding=True`), 
- очень длинные последовательности (с усечением `truncation=True`) и 
- выбором типа тензора (`return_tensors="pt"`)  

и подаем в модель:

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

model_inputs_2 = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output_2 = model(**model_inputs_2)
output_2.logits

tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>)

`truncation=True` - обрезает текст, если он слишком длинный для модели.  
Большинство моделей ограничены по длине входных данных. Например:  
- bert-base-uncased → максимум 512 токенов  
- distilbert-base-uncased → максимум 512 токенов  
- gpt-2 → максимум 1024 токена  
Если текст длиннее, модель не сможет его обработать, и возникнет ошибка.  

Можно задать конкретную длину:  
```
tokens = tokenizer(long_text, truncation=True, max_length=128)
print(len(tokens["input_ids"]))  # 128
```
По умолчанию, truncation=True обрезает текст справа (right truncation).  
Можно изменить стратегию:  
`tokens = tokenizer(long_text, truncation=True, truncation_side="left")  # Обрежет слева`  

__Итог: Как правильно использовать truncation?__  
```
Ситуация	                                 Как использовать?
Просто обрезать слишком длинный текст	         truncation=True ✅
Обрезать до фиксированного количества токенов    truncation=True, max_length=128 ✅
Обрезать слева, а не справа	                 truncation=True, truncation_side="left" ✅
```


Наконец, нас интересуют не логиты, а вероятности правильного/неправильного ответа модели

In [11]:
import torch

predictions = torch.nn.functional.softmax(output_2.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [5.3534e-04, 9.9946e-01]], grad_fn=<SoftmaxBackward0>)


Формула вычисления вероятности:  
$$
P_i = \frac{e^{x_i}}{\sum e^{x_j}}
$$


Допустим, у нас есть логиты:

$$
x = [2.1, 1.2]
$$

Вычисляем экспоненту:

$$
e^{2.1} \approx 8.16, \quad e^{1.2} \approx 3.32
$$

Считаем сумму:

$$
\sum e^{x_j} = 8.16 + 3.32 = 11.48
$$

Теперь вероятность каждого класса:

$$
P_1 = \frac{8.16}{11.48} \approx 0.71, \quad
P_2 = \frac{3.32}{11.48} \approx 0.29
$$